In [1]:
import os

In [2]:
%pwd

'd:\\Piyush\\End-to-end-classification-project-pipeline-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Piyush\\End-to-end-classification-project-pipeline-'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

[2024-05-06 08:37:02,999: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.el
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [12]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    '''This method loads a pre-trained model from the path specified in the updated_base_model_path attribute of the config.
      It stores the loaded model in the model attribute of the Training instance.'''
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )


    '''This method prepares data generators for both training and validation data.'''
    def train_valid_generator(self):

        '''This line creates a dictionary datagenerator_kwargs with parameters for data augmentation and validation split.'''
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        '''This line creates a dictionary dataflow_kwargs with parameters related to data flow such as target size, batch size, and interpolation method.'''
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        '''This line creates an ImageDataGenerator instance for validation data with the provided augmentation parameters.'''
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        '''This line generates a data generator (valid_generator) for validation data by flowing images from the directory specified in training_data attribute of config.'''
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        '''This block checks if data augmentation is enabled (params_is_augmentation attribute in config). If it's enabled, it creates a new ImageDataGenerator instance (train_datagenerator) for training data with augmentation parameters. If not, it uses the same generator as for validation data.'''
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        '''This line generates a data generator (train_generator) for training data by flowing images from the directory specified in training_data attribute of config.'''
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    '''This is a static method that saves a Keras model to the specified path.'''
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    '''This method prepares for training by calculating the number of steps per epoch and validation steps based on the number of samples and batch size.'''
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        '''This line trains the model using the training data generator (train_generator) for a specified number of epochs, 
        with the given number of steps per epoch, and validation steps. It uses the validation 
        data generator (valid_generator) for validation during training and applies any provided callbacks.'''
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        '''Finally, this line saves the trained model to the specified path using the static save_model method.'''
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-06-01 10:18:59,787: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-01 10:18:59,792: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-01 10:18:59,794: INFO: common: created directory at: artifacts]
[2023-06-01 10:18:59,795: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-06-01 10:18:59,796: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-06-01 10:18:59,797: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
19/19 [==============================] - 28s 1s/step - loss: 13.6681 - accuracy: 0.5270 - val_loss: 19.1089 - val_accuracy: 0.3906
